## OpenAI API 와 허깅페이스 오픈소스 LLM

* 영상 튜토리얼 : https://youtu.be/pjAPCqJIOXI

<img src='https://raw.githubusercontent.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs/refs/heads/main/cover.png'
     alt="NLP와 LLM 실전 가이드(한빛미디어)"
     style="border: 3px solid gray; box-shadow: 5px 5px 15px rgba(0, 0, 0, 0.3); border-radius: 10px; width: 300px;"   width='300'>"  width="300">

* 저자:  
    - [Lior Gazit](https://www.linkedin.com/in/liorgazit).  
    - [Meysam Ghaffari](https://www.linkedin.com/in/meysam-ghaffari-ph-d-a2553088/).
* 역자:
    - [박조은](https://github.com/corazzon)
* 이 노트북은 다음의 책에서 소개하는 내용입니다.
    - 역서 : NLP와 LLM 실전 가이드(한빛미디어)
    - 원서 : [Mastering NLP from Foundations to LLMs](https://www.amazon.com/dp/1804619183)

**이 노트북의 목적:**  
이 노트북에서는 코드를 통해 LLM 프롬프팅을 설정하는 방법을 시연합니다.  
먼저 OpenAI의 GPT API를 사용하는 실험을 진행하고,  
그다음 오픈소스 LLM을 로컬 환경에 가져와서 실험을 진행합니다.

**요구사항:**  
* Google Colab에서 실행 시, 다음 런타임 노트북 설정을 사용하세요: `Python3, CPU`  
* 노트북의 모든 기능을 활용하기 위해서는 OpenAI API 사용을 권장합니다

colab 실습 :
https://github.com/corazzon/Mastering-NLP-from-Foundations-to-LLMs

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/Mastering-NLP-from-Foundations-to-LLMs/blob/main/Chapter8_notebooks/Ch8_Setting_Up_Close_Source_and_Open_Source_LLMs.ipynb)  


원서 Colab notebook의 Github repo:
https://github.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs   


<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/blob/liors_branch/Chapter8_notebooks/Ch8_Setting_Up_Close_Source_and_Open_Source_LLMs.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

>*```면책사항: 이 노트북에서 다루는 내용과 아이디어는 저자들 개인의 것이며, 저자들의 고용주의 견해나 지적 재산을 대변하지 않습니다.```*

## Part 1 - API를 통한 파이썬에서의 LLM 사용
이 파트에서는 OpenAI의 API를 통해 `gpt-4o-mini`에 접근하기 위한 설정 과정을 다룹니다.  
책에서 설명한 단계들을 차례대로 살펴볼 것입니다.  
> **OpenAI 웹사이트에서 API 키를 미리 생성해야 하며, 계정을 설정하면 API 키를 받을 수 있습니다.**  
API 키가 없는 경우, Part 2로 넘어가서 무료 오픈소스 LLM 설정 방법을 확인하세요.

설치:

In [ ]:
# 주의사항:
# 아래 코드에서 Python 패키지 불일치로 인한 오류가 발생하는 경우, 새로운 버전이 원인일 수 있습니다.
# 이런 경우, "default_installations"를 False로 설정하여 원래 이미지로 되돌리세요:
default_installations = True
if default_installations:
    !pip -q install --upgrade openai
else:
    import requests
    text_file_path = "requirements__Ch8_Setting_Up_Close_Source_and_Open_Source_LLMs.txt"
    url = "https://raw.githubusercontent.com/PacktPublishing/Mastering-NLP-from-Foundations-to-LLMs/main/Chapter8_notebooks/" + text_file_path
    res = requests.get(url)
    with open(text_file_path, "w") as f:
        f.write(res.text)

    !pip install -r requirements__Ch8_Setting_Up_Close_Source_and_Open_Source_LLMs.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 6.7 MB/s eta 0:00:00


Imports:

In [ ]:
import openai
import re
import time

OpenAI API key:  
**문자열 형태로 아래 "..."에 OpenAI에서 발급받은 key를 입력해 주세요!**  


Colab 보안 비밀 설정은 왼쪽 열쇠 모양의 아이콘을 클릭하면 나옵니다.
<img src="https://i.imgur.com/7P383n4.png" width="500">




OpenAI API 키 정의:  
**키를 제공하고 문자열로 입력해야 합니다!**  
키가 없는 경우, Part 2로 넘어가 무료로 오픈 소스 LLM을 설정하는 방법을 확인하세요.  

In [ ]:
# paste your key here as a string: "..."
api_key = "..."

In [ ]:
# colab 사용시 주석 해제 후 사용
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
client = openai.OpenAI(api_key=api_key)
client

### `gpt-4o-mini` 사용해 보기
* https://platform.openai.com/docs/overview

In [ ]:
# 프라이밍 예시
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system",
         "content": "당신은 도움이 되는 어시스턴트입니다. 간단한 답변을 제공하며, 답변은 Markdown 문법으로 포맷합니다."},
        {"role": "user",
         "content": "Python 라이브러리 pandas를 어떻게 임포트하나요?"},
        {"role": "assistant",
         "content": "다음과 같이 pandas를 임포트할 수 있습니다:  \n```\nimport pandas as pd\n```"},
        {"role": "user",
         "content": "Python 라이브러리 numpy를 어떻게 임포트하나요?"}
    ])
text = response.choices[0].message.content.strip()
print(text)

다음과 같이 NumPy를 임포트할 수 있습니다:  
```python
import numpy as np
```


In [ ]:
response.choices[0].message.content.strip()

'다음과 같이 NumPy를 임포트할 수 있습니다:  \n```python\nimport numpy as np\n```'

코드 설정:

In [ ]:
openai_model = "gpt-4o-mini"
temperature = 0
max_attempts = 5
attempts = 0

#### 프롬프트 정의하기
여기서는 모델에게 전달할 두 가지 메시지를 정의합니다:
1. 시스템 프롬프트: 모델에게 원하는 동작 방식을 알려주는 프라이밍입니다.
   웹 브라우저의 ChatGPT 인터페이스에서는 제공되지 않는 기능입니다.
2. 사용자 프롬프트.

참고:  
OpenAI는 더 정교한 프라이밍을 위한 추가 옵션들을 제공합니다.

In [ ]:
# system_prompt = "Your are an insightful assistant. When you are asked a question, you articulate an answer. Only after you have finished answering the question, you carefully review all the words in the prompt and you state the typos that exist in the prompt, finally, you provide corrections to them."
# user_prompt_oai = "If neuroscience could extract the last thoughts a person had before they dyed, how would the world be diferent?"

In [ ]:
system_prompt = "당신은 통찰력 있는 어시스턴트입니다. 질문을 받으면 답변을 명확하게 설명합니다. 답변을 마친 후에는 프롬프트의 모든 단어를 주의 깊게 검토하여 오타를 찾아내고, 마지막으로 해당 오타들에 대한 수정 사항을 제시합니다."
# 의도된 오타 (주출=>추출, 달라질까요?=>달아질까요?)
user_prompt_oai = "신경과학이 사람이 죽기 전 마지막 생각을 주출할 수 있다면, 세상은 어떻게 달아질까요?"

프라이밍(priming) 메시지를 다음과 같이 정의합니다:

In [ ]:
# 메시지 구성하기:
messages = []
messages.append({"role": "system",
                "content": system_prompt})
messages.append({"role": "user",
                "content": user_prompt_oai})

#### 모델 실험하기

In [ ]:
while True:
    try:
        response = client.chat.completions.create(
            model=openai_model,
            messages=messages,
            temperature=temperature)
        response_oai = response.choices[0].message.content.strip()
        response_oai = re.sub(r'\. ', r'. \n', response_oai)
        print(f"프롬프트: {user_prompt_oai}\n\n{openai_model}'의 응답: \n{response_oai}")
        break
    except Exception as output:
        attempts += 1
        if attempts >= max_attempts:
            print(f"Quitting due to {openai_model} error: {output}")
            break
        print(f"Attempt #{attempts} failed: {output}")
        time.sleep(1)

프롬프트: 신경과학이 사람이 죽기 전 마지막 생각을 주출할 수 있다면, 세상은 어떻게 달아질까요?

gpt-4o-mini'의 응답: 
신경과학이 사람이 죽기 전 마지막 생각을 추출할 수 있다면, 세상은 여러 면에서 크게 변화할 수 있습니다. 


1. 
**윤리적 논의**: 마지막 생각을 추출하는 기술이 개발된다면, 이는 개인의 사생활과 윤리에 대한 심각한 논의를 촉발할 것입니다. 
사람의 마지막 생각이 공개되거나 기록되는 것이 개인의 권리를 침해하는지에 대한 논의가 필요할 것입니다.

2. 
**의료 및 심리학적 접근**: 의사와 심리학자들은 환자의 마지막 생각을 통해 그들의 감정이나 두려움을 이해하고, 이를 바탕으로 더 나은 치료 방법이나 심리적 지원을 제공할 수 있을 것입니다.

3. 
**사회적 변화**: 사람들이 죽음을 맞이하는 방식이나 죽음에 대한 인식이 변화할 수 있습니다. 
마지막 생각이 공유되면, 죽음에 대한 두려움이나 불안이 줄어들 수 있으며, 죽음을 더 자연스럽고 개방적으로 받아들이는 사회가 될 수 있습니다.

4. 
**기억과 유산**: 마지막 생각이 기록되면, 고인의 기억이나 유산을 보존하는 새로운 방법이 생길 수 있습니다. 
이는 가족이나 친구들에게 큰 의미가 있을 수 있습니다.

5. 
**과학적 연구**: 마지막 생각을 연구하는 과정에서 인간의 의식, 기억, 감정에 대한 새로운 통찰이 생길 수 있으며, 이는 신경과학의 발전에 기여할 수 있습니다.

이러한 변화들은 긍정적일 수도, 부정적일 수도 있으며, 사회 전반에 걸쳐 깊은 영향을 미칠 것입니다.

**오타 검토 및 수정 사항**:
- "주출" → "추출" (올바른 단어는 "추출"입니다.)
- "달아질까요" → "달라질까요" (올바른 단어는 "달라질까요"입니다.) 

수정된 문장: "신경과학이 사람이 죽기 전 마지막 생각을 추출할 수 있다면, 세상은 어떻게 달라질까요?"


****

## Part 2 - 로컬 환경에서 오픈소스 LLM 사용하기
이번에는 Hugging Face 허브를 통해 오픈소스 LLM을 로컬 환경에 설정하는 방법을 알아보겠습니다.

설치:

In [ ]:
%pip -q install --upgrade transformers

Imports:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

### 마이크로소프트의 `DialoGPT-medium` 사용해 보기
* https://huggingface.co/microsoft/DialoGPT-medium

설정:

In [ ]:
hf_model = "microsoft/DialoGPT-medium"
max_length = 1000

tokenizer = AutoTokenizer.from_pretrained(hf_model)
model = AutoModelForCausalLM.from_pretrained(hf_model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

#### 프롬프트 정의

In [ ]:
user_prompt_hf = "If dinosaurs were alive today, would they possess a threat to people?"
# user_prompt_hf = "만약 공룡들이 오늘날 살아있다면, 그들은 인류에게 위협이 될까요? 한국어로 답변해 주세요"

#### 모델 사용해 보기

In [ ]:
print("\n* 참고사항: 위에 표시된 모델 다운로드 경고는 무시하셔도 됩니다.")
user_input_ids = tokenizer.encode(user_prompt_hf + tokenizer.eos_token, return_tensors='pt')
response_hf_encoded = model.generate(user_input_ids,
                             max_length=max_length,
                             pad_token_id=tokenizer.eos_token_id)
response_hf = tokenizer.decode(response_hf_encoded[:, user_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"\n\n프롬프트: {user_prompt_hf}\n\n{hf_model}의 응답: \n{response_hf}")


* 참고사항: 위에 표시된 모델 다운로드 경고는 무시하셔도 됩니다.


프롬프트: 만약 공룡들이 오늘날 살아있다면, 그들은 인류에게 위협이 될까요? 한국어로 답변해 주세요

microsoft/DialoGPT-medium의 응답: 
I'm not sure if you're being serious or not.


### 한국어 지원 모델 사용하기
* https://huggingface.co/lcw99/ko-dialoGPT-korean-chit-chat

In [ ]:
hf_model = "lcw99/ko-dialoGPT-korean-chit-chat"
max_length = 1000

tokenizer = AutoTokenizer.from_pretrained(hf_model)
model = AutoModelForCausalLM.from_pretrained(hf_model)


user_prompt_hf = "만약 공룡들이 오늘날 살아있다면, 그들은 인류에게 위협이 될까요?"

print("\n* 참고사항: 위에 표시된 모델 다운로드 경고는 무시하셔도 됩니다.")
user_input_ids = tokenizer.encode(user_prompt_hf + tokenizer.eos_token, return_tensors='pt')
response_hf_encoded = model.generate(user_input_ids,
                             max_length=max_length,
                             pad_token_id=tokenizer.eos_token_id)
response_hf = tokenizer.decode(response_hf_encoded[:, user_input_ids.shape[-1]:][0], skip_special_tokens=True)
print(f"\n\n프롬프트: {user_prompt_hf}\n\n{hf_model}의 응답: \n{response_hf}")

tokenizer_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]


* 참고사항: 위에 표시된 모델 다운로드 경고는 무시하셔도 됩니다.


프롬프트: 만약 공룡들이 오늘날 살아있다면, 그들은 인류에게 위협이 될까요?

lcw99/ko-dialoGPT-korean-chit-chat의 응답: 
네그럴거같아요. ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ

* 다른 한국어 모델 찾아보기 : https://huggingface.co/models?search=korean